In [1]:
print("Hi Hello. How are you all?")

Hi Hello. How are you all?


In [2]:
%pwd

'c:\\Users\\gouds\\Desktop\\medibot\\research'

In [3]:
import os
os.chdir("../")

In [ ]:
#it should be c:\\Users\\gouds\\Desktop\\medibot 
%pwd

'c:\\Users\\gouds\\Desktop\\medibot'